# Koopman operator model predictive control for flow control

In this tutorial we use model predictive control to steer a flow simulation. For this we repeat the results in a similar setting of the paper (Section 5.1) and attached repository.

* Paper: "A data-driven Koopman model predictive control framework for nonlinear flows" https://arxiv.org/pdf/1804.05291.pdf
* Supplementary code: https://github.com/arbabiha/KoopmanMPC_for_flowcontrol

Note that there are discrepancies between the code and the description in the paper. We mostly align to the setting to the code and highlight its discrepancies to the paper.

The model to be controlled is a one-dimensional Burger equation

$$
\frac{\partial v}{\partial t} + v \frac{\partial v}{\partial x} = \nu \frac{\partial v^2}{\partial^2 x} + f(x,t)
$$

with space and time domains

$$
x \in [0, 2\pi] \\
t \in [0, \infty]
$$

and periodic boundary conditions $v(0, t) = v(2\pi, t)$.

In the paper the domain is $x \in [0, 1]$, while our choice matches the setting in the code of the repository.

There are two control parameters $u=(u_1, u_2)$ with constrains $-0.1 \leq u_{1,2} \leq 0.1$. With these we can steer the system state to a reference time series. The control parameters affect the state with:

$
\begin{align}
f(x,t) &= u_1(t) f_1(x) + u_2(t)f_2(x)\\
&= u_1(t) \exp(-((15/(2\pi) (x - \pi/2))^2)) + u_2(t) \exp(-((15/(2\pi) (x - 3/2\pi))^2)))
\end{align}
$

The function parameters are adapted to the larger domain $x \in [0, 2\pi]$, which diverges to the description in the paper.

We specify an initial condition of the system with a functional relation

$$
v(x, 0) = a \exp(-(((x - \pi) 5/(2\pi))^2)) + (1 - a) \sin(2x)^2
$$

where we uniformly sample $a \in (0,1)$.

The control objective is to follow the reference state, which changes over time in a piecewise function

$
\begin{align}
v_{ref} (x, 0 \leq t \leq 20) &= 0.5\\
v_{ref} (x, 20 \leq t \leq 40) &= 1\\
v_{ref} (x, 40 \leq t \leq 60) &= 0.5
\end{align}
$

In the paper the time frequency is stated as 0.01 to integrate the system. However, in the final data collection in the code every 19th state is actually taken. This leads to an effective time interval of $\Delta t = 0.19$. Therefore, the time intervals in the reference $v_{ref}$ are adapted such that they roughly match the setting in the paper.

To obtain a suitable control sequence $u$ to steer an initial condition towards $v_{ref}$, we use Koopman-based Model Predictive Control (KMPC). After we sample example data, we identify the system dynamics with the Extended Dynamic Mode Decomposition (EDMD).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from scipy.interpolate import interp1d
from sklearn.base import BaseEstimator
from tqdm import tqdm

from datafold import (
    EDMD,
    DMDControl,
    TSCColumnTransformer,
    TSCDataFrame,
    TSCIdentity,
    TSCTakensEmbedding,
    TSCTransformerMixin,
)
from datafold.appfold.mpc import LinearKMPC
from datafold.utils._systems import Burger1DPeriodicBoundary

In [ ]:
rng = np.random.default_rng(2)

## Data collection from the original system

Set up Burger system, control functions (f1 and f2) and the initial condition function. We sample the full Burger system, but for EDMD we only use every 10th spatial point in the domain.

In [ ]:
sys = Burger1DPeriodicBoundary(n_spatial_points=100, nu=0.01)

# control function
f1 = np.atleast_2d(np.exp(-((15 / (2 * np.pi) * (sys.x_nodes - 0.5 * np.pi)) ** 2)))
f2 = np.atleast_2d(np.exp(-((15 / (2 * np.pi) * (sys.x_nodes - 1.5 * np.pi)) ** 2)))

# initial condition function
ic1 = np.exp(-(((sys.x_nodes - 2 * np.pi * 0.5) * 5 / (2 * np.pi)) ** 2))
ic2 = np.sin(2 * sys.x_nodes) ** 2
icfunc = lambda a: a * ic1 + (1 - a) * ic2

In [ ]:
# sampling options
dt = 0.19  # time frequency
sim_length = 200  # time series length
training_size = 100  # number of initial conditions to sample


# function to subselect states to every 10th spatial point
def subselect_measurements(tscdf):
    return tscdf.iloc[:, 9::10]


# contains on the effective control parameters
umin, umax = (-0.1, 0.1)

# time values of a single time series
time_values = np.arange(0, dt * sim_length + 1e-12, dt)

### Exectue system sampling

Fill lists of time series by resetting the the initial condition with $a \sim \operatorname{Uniform(0,1)}$. The final data is then captured in a single `TSCDataFrame` as *datafold*'s main data structure.

In [ ]:
# lists to collect both time series and control
X_tsc = []
U_tsc = []

for i in tqdm(range(training_size)):
    # sample a new initial condition
    ic = icfunc(rng.uniform(0, 1))

    # sample random control parameters (for each timestep) over the simulation horizon
    # describe the control input as a function f(t, x)
    rand_vals = rng.uniform(umin, umax, size=(len(time_values), 2))
    U1rand = lambda t: np.atleast_2d(
        interp1d(time_values, rand_vals[:, 0], kind="previous")(t)
    ).T
    U2rand = lambda t: np.atleast_2d(
        interp1d(time_values, rand_vals[:, 1], kind="previous")(t)
    ).T

    def f(t, x):
        # while we have 2 effective control inputs they affect each state coordinate according to this function
        return U1rand(t) * f1 + U2rand(t) * f2

    # perform system prediction
    X_predict, Ufull = sys.predict(
        ic, U=f, time_values=time_values, require_last_control_state=False
    )

    # drop last control input, because for the last state no prediction is performed
    U = TSCDataFrame.from_array(
        rand_vals[:-1, :],
        time_values=Ufull.time_values(),
        feature_names=["u1", "u2"],
    )

    X_tsc.append(X_predict)
    U_tsc.append(U)

# finalize the time series collection data to be used for system identification with control input
X_tsc = TSCDataFrame.from_frame_list(X_tsc)
U_tsc = TSCDataFrame.from_frame_list(U_tsc)

Snapshot of sampled system states and control input.

In [ ]:
print(X_tsc.n_timeseries)
X_tsc

In [ ]:
U_tsc

Animate sampled time series data with control input. 

In [ ]:
tsid = 0  # select time series ID to plot

f, ax = plt.subplots(figsize=(8, 7), nrows=2)
plt.close()  # close to perform video animation, the figure still exists

(ref_line,) = ax[0].plot(
    sys.x_nodes, X_tsc.loc[pd.IndexSlice[tsid, :], :].iloc[0].to_numpy(), label="model"
)
ax[0].legend(loc="upper left")


def Ufunc(u):
    return u[0] * f1 + u[1] * f2


(control_line,) = ax[1].plot(
    sys.x_nodes,
    Ufunc(U_tsc.loc[pd.IndexSlice[tsid, :], :].iloc[0].to_numpy()).ravel(),
    label="randomly sampled control input",
)
ax[1].set_ylim(-0.1, 0.1)
ax[1].legend(loc="upper left")


def func(i):
    ref_line.set_ydata(X_tsc.loc[pd.IndexSlice[tsid, :], :].iloc[i, :].to_numpy())
    vals = U_tsc.loc[pd.IndexSlice[tsid, :], :].iloc[i, :].to_numpy()
    control_line.set_ydata(Ufunc(vals))
    return (
        ref_line,
        control_line,
    )


anim = FuncAnimation(f, func=func, frames=U_tsc.n_timesteps)
HTML(anim.to_html5_video())

## Set up system identification and control input using Extended Dynamic Mode Decomposition 

For the system identification we adapt the sampled data as follows:

* We only use a reduced number of spatial points (every 10th grid point), which reduces the dimensionality of the system.
* We attach the control input in $U$ to the system states in $X$. For this, we also shift the time index in $U$ by one such that the (past) control input is attached to the actual resulting system state. Because of this time shift the initial system state has no matching control input. We fill this with zeros, however, these values are ignored later when performing a time delay embedding in the EDMD dictionary.

**Note:**

In EDMD the system states and control input are treated separately. This means attaching the control input to the system states is only done to enrich the system state and not a mandatory step when using EDMD with control.  

In [ ]:
def shift_time_index_U(_X, _U):
    new_index = _X.groupby("ID").tail(_X.n_timesteps - 1).index
    return _U.set_index(new_index)

In [ ]:
# use only selected spatial points
X_tsc_reduced = subselect_measurements(X_tsc)

# attach control input to system state
X_tsc_reduced = pd.concat(
    [X_tsc_reduced, shift_time_index_U(X_tsc_reduced, U_tsc)], axis=1
)

# fill nan values with 0 where no corresponding control input was available
X_tsc_reduced = X_tsc_reduced.fillna(0)

Display effective system states used within EDMD

In [ ]:
X_tsc_reduced

## Setting up EDMD with dictionary and underlying DMD model

We now set up the EDMD dictionary (as a pipeline). First we specify a custom data transformation to compute the L2 norm from the sates (as described in paper).

In [ ]:
class L2Norm(BaseEstimator, TSCTransformerMixin):
    def fit(self, X):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["L2norm"]

    def transform(self, X: TSCDataFrame, y=None):
        return TSCDataFrame.from_same_indices_as(
            X,
            np.sum(np.square(np.abs(X.to_numpy())), axis=1) / X.shape[1],
            except_columns=self.get_feature_names_out(),
        )

Now we describe the dictionary, where we compute the L2-norm on the system states, time delay embedding and also add a constant vector.  

In [ ]:
l2norm = ("l2_x", L2Norm(), lambda df: df.columns.str.startswith("x"))

delay1 = (
    "delay_x",
    TSCTakensEmbedding(delays=4),
    lambda df: df.columns.str.startswith("x"),
)
delay2 = (
    "delay_u",
    TSCTakensEmbedding(delays=3),
    lambda df: df.columns.str.startswith("u"),
)

_dict = (
    "tde",
    TSCColumnTransformer([l2norm, delay1, delay2], verbose_feature_names_out=False),
)

_id = ("_id", TSCIdentity(include_const=True))

dict_steps = [_dict, _id]

# It is essential to use DMD with control in EDMD.
# The flag `dict_preserves_id_states=True` indicates that the original states are contained in the dictionary,
# which makes the inverse mapping from dictionary states to full states easier as it is only a projection.
edmd = EDMD(
    dict_steps,
    dmd_model=DMDControl(),
    include_id_state=False,
    dict_preserves_id_state=True,
)

# display html representation of object
edmd

With the specified EDMD model, we can now fit the model with the sampled data comprising states `X` and control input `U`.

In [ ]:
edmd.fit(X_tsc_reduced, U=U_tsc);

We can now look at the time series data in the dictionary space. Note that the first samples of the time series are dropped due to the time delay embedding. The number of samples in `X` necessary to map from full-state to dictionary state is available in the attribute `edmd.n_samples_ic_`. 

In [ ]:
print(f"{edmd.n_samples_ic_=}")
edmd.transform(X_tsc_reduced)

## Set up control optimization with model predictive control

Prediction parameters:

In [ ]:
horizon = 20  # the horizon is the number of steps
Tpred = dt * horizon  # prediction horizon in MPC
Tend = 70  # end time to predict time series
Nsim = int(Tend // dt) + 1  # number of simulation steps in MPC loop

Because we need 5 states to map to a dictionary space, we first perform a warm-up phase where we evaluate the system with no control applied.

In [ ]:
ic = icfunc(0.2)  # can also be adapted to control another time series

X_init, _ = sys.predict(
    ic,
    # Why edmd.n_samples_ic_-1 in U:
    # -> typically no control input is needed for the final system state
    U=np.zeros((edmd.n_samples_ic_ - 1, sys.n_control_in_)),
    time_values=np.arange(0, edmd.n_samples_ic_ * dt, dt),
)

Set up `LinearKMPC` model which will optimize for control sequence.  

In [ ]:
kmpc = LinearKMPC(
    edmd=edmd,
    horizon=horizon,
    input_bounds=np.array([[-0.1, 0.1], [-0.1, 0.1]]),
    state_bounds=None,
    qois=X_tsc_reduced.columns[X_tsc_reduced.columns.str.startswith("x")],
    cost_running=1,
    cost_terminal=1,
    cost_input=1,
)

Generate the reference time series (both in full and reduced coordinates).

In [ ]:
start_time = X_init.time_values()[-1]
time_values_ref = np.arange(0, start_time + Tend, dt)

X_ref = np.zeros(len(time_values_ref))
X_ref[time_values_ref <= 20] = 0.5
X_ref[np.logical_and(time_values_ref > 20, time_values_ref < 40)] = 1
X_ref[time_values_ref > 40] = 0.5
X_ref = np.outer(X_ref, np.ones(X_tsc.shape[1]))
X_ref = TSCDataFrame.from_array(
    X_ref, time_values=time_values_ref, feature_names=X_tsc.columns
)

X_ref_reduced = subselect_measurements(X_ref)

U_ic = TSCDataFrame.from_array(
    np.zeros((edmd.n_samples_ic_ - 1, 2)),
    time_values=X_init.time_values()[:-1],
    feature_names=edmd.control_names_in_,
)

X_ref_reduced

In [ ]:
# keep track of current system and EDMD state
model_state = X_init.iloc[[-1], :]
edmd_state = pd.concat(
    [subselect_measurements(X_init), shift_time_index_U(X_init, U_ic)], axis=1
).fillna(0)

# recotrd the model evolution and optimized control input
X_model_evolution = X_init
U_evolution = U_ic

# record the uncontrolled time series for comparison
X_model_unctr_evolution = X_init.copy()

# record system error between state and reference time series
X_error_evolution = X_init - X_ref.iloc[[0], :].to_numpy()

for i in tqdm(range(X_init.shape[0], Nsim)):
    # reference time series over horizon
    reference = X_ref_reduced.iloc[i : i + horizon, :]

    if reference.shape[0] != kmpc.horizon:
        # stop loop if the rest of reference signal is smaller than horizon
        break

    t = X_model_evolution.time_values()[-1]
    t_new = X_model_evolution.time_values()[-1] + dt

    # optimize the control input on EDMD
    U = kmpc.control_sequence(edmd_state, reference=reference)

    # use only the first control input for the next step
    Ufull = U.iloc[0, 0] * f1 + U.iloc[0, 1] * f2
    U_evolution = pd.concat([U_evolution, U.iloc[[0], :]], axis=0)

    # apply the obtained control input from EDMD to the actual system
    X_model, _ = sys.predict(
        X_model_evolution.iloc[[-1], :], U=Ufull, time_values=np.array([t, t_new])
    )
    X_model_evolution = pd.concat([X_model_evolution, X_model.iloc[[1], :]], axis=0)

    # record difference between model and reference
    diff = X_model.iloc[[1], :] - X_ref.iloc[[i], :].to_numpy()
    X_error_evolution = pd.concat([X_error_evolution, diff])

    # perform separate uncontrolled system
    X_model_unctr, _ = sys.predict(
        X_model_unctr_evolution.iloc[[-1], :],
        U=np.zeros_like(sys.x_nodes)[np.newaxis, :],
        time_values=np.array([t, t_new]),
    )
    X_model_unctr_evolution = pd.concat(
        [X_model_unctr_evolution, X_model_unctr.iloc[[1], :]], axis=0
    )

    # prepare new edmd_state for next iteration (attach the shifted control input)
    X_model_last = subselect_measurements(
        X_model_evolution.iloc[-edmd.n_samples_ic_ :, :]
    )
    U_last = U_evolution.iloc[-edmd.n_samples_ic_ : -1, :]
    U_last_shifted = shift_time_index_U(
        X_model_last, shift_time_index_U(X_model_last, U_last)
    )
    edmd_state = pd.concat([X_model_last, U_last_shifted], axis=1).fillna(0)

Animate the controlled and uncontrolled system. Note that because of the state prediction, the control input already changes before `t=20`, where the reference state is changed from $v_{ref}=0.5$ to $v_{ref}=1$. 

In [ ]:
f, ax = plt.subplots(figsize=(8, 8), nrows=3, sharex=True)
plt.close()  # see https://stackoverflow.com/a/47138474 (first comment)
(model_line,) = ax[0].plot(
    sys.x_nodes, X_model_evolution.iloc[0], label="system controlled"
)
(model_uctr_line,) = ax[0].plot(
    sys.x_nodes, X_model_unctr_evolution.iloc[0], label="system uncontrolled"
)
(ref_line,) = ax[0].plot(sys.x_nodes, X_ref.iloc[0], label="reference")
ax[0].legend(loc="upper left")
ax[0].set_ylim(0, 1.3)

Ufunc = lambda u, x: (u[0] * f1 + u[1] * f2).ravel()
(control_line,) = ax[1].plot(
    sys.x_nodes,
    Ufunc(U_evolution.iloc[0, :].to_numpy(), None),
    label="control",
)
ax[1].set_ylim(umin, umax)
ax[1].legend(loc="upper left")

(error_line,) = ax[2].plot(
    sys.x_nodes, X_error_evolution.iloc[0, :].to_numpy(), c="red", label="difference"
)
ax[2].legend(loc="upper left")


def func(i):
    model_line.set_ydata(X_model_evolution.iloc[i, :].to_numpy())
    model_uctr_line.set_ydata(X_model_unctr_evolution.iloc[i, :].to_numpy())
    ref_line.set_ydata(X_ref.iloc[i, :].to_numpy())
    control_line.set_ydata(Ufunc(U_evolution.iloc[i].to_numpy(), None))
    error_line.set_ydata(X_error_evolution.iloc[i, :].to_numpy())

    return (model_line, model_uctr_line, ref_line, error_line)


anim = FuncAnimation(f, func=func, frames=U_evolution.shape[0])
HTML(anim.to_html5_video())
